In [1]:
#quickstart
from pathlib import Path
import pandas as pd
from bids import BIDSLayout
from joblib import Parallel, delayed

import numpy as np
import matplotlib.pylab as plt

import rpy2
from rpy2.robjects.packages import importr
from pymer4 import Lmer
psych = importr('psych')


import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

from rpy2.robjects.conversion import localconverter

import os
import glob
import seaborn as sns
from nltools.data import Brain_Data, Design_Matrix, Adjacency
from nltools.mask import expand_mask, roi_to_brain
from nltools.stats import zscore
from scipy.stats import binom, ttest_1samp
from sklearn.metrics import pairwise_distances
from nltools.stats import fdr, one_sample_permutation
from copy import deepcopy
from nltools.utils import get_anatomical
import nltools.plotting as nlplt
from datetime import date
from joblib import Parallel, delayed

from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

from nilearn.input_data import NiftiMapsMasker,NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure

import sys
import pandas as pd
import rpy2
from rpy2.robjects.packages import importr
from pymer4 import Lmer
psych = importr('psych')
import numpy as np

import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

from rpy2.robjects.conversion import localconverter


%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 1000)

from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))

PermissionError: [Errno 13] Permission denied: '/gpfs/gsfs11/users/MBDU/midla/env/lib/python3.7/site-packages/pyflakes-2.2.0.dist-info'

In [48]:
#Converts BIDS formatted jsons into jsons with single trial beta outputs
#Takes original json path, target json directory path, a list of target conditions, and the number of total trials
#Generates json for each trial of each condition with single trial beta output
def single_trial_beta_model(json_orig_path, json_new_path, condition_names, num_trials):
    for trial_n in range(num_trials):
        for condition_name in condition_names:
            with open(json_orig_path, 'r') as h:
                jdat = json.load(h)
                
            #Replaces target condition with target trial and other trials regressors
            jdat["Steps"][0]['Model']['X'].remove(condition_name)
            jdat["Steps"][0]['Model']['X'].append(f"trial-{trial_n:03d}_{condition_name}")
            jdat["Steps"][0]['Model']['X'].append(f"other_trials_{condition_name}")
            #Removes all model levels other than 1st level; contrasts, and existing dummy contrasts
            jdat['Steps'] = [jdat['Steps'][0]]
            jdat['Steps'][0].pop('Contrasts', None)
            jdat['Steps'][0].pop('DummyContrasts', None)
            
            #Adds target trial regressors to convolve list
            to_convolve = [x for x in jdat["Steps"][0]['Transformations'] if x["Name"] == "Convolve"][0]["Input"]
            to_convolve.remove(condition_name)
            to_convolve.append(f"trial-{trial_n:03d}_{condition_name}")
            to_convolve.append(f"other_trials_{condition_name}")
            [x for x in jdat["Steps"][0]['Transformations'] if x["Name"] == "Convolve"][0]["Input"] = to_convolve
            
            #Adds target trial regressor to dummy contrast
            jdat['DummyContrasts'] = {
                "Conditions": [f"trial-{trial_n:03d}_{condition_name}"],
                "Type":"t"
            }
            json_new_path = f'{json_new_directory}stb_trial-{trial_n:03d}_{condition_name}.json'
            with open(json_new_path, 'w') as h:
                json.dump(jdat, h, indent=2)

In [2]:
json_orig_path = '/data/MBDU/midla/data/derivatives/fitlins/models/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-20900/ses-01/run-1/sub-20900_ses-01_task-mid_run-1_desc-model.json'
json_new_path = '/data/MBDU/midla/notebooks/campcc/single_trial_betas/test_file2.json'
condition_names = ['gain_anticipation']
num_trials = 2
#single_trial_beta_model(json_orig_path, json_new_path, condition_names, num_trials)

In [44]:
#json_orig_path = '/data/MBDU/midla/data/derivatives/fitlins/models/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-20900/ses-01/run-1/sub-20900_ses-01_task-mid_run-1_desc-model.json'
json_orig_path = '/data/MBDU/midla/notebooks/campcc/single_trial_betas/test_file.json'
json_new_path = '/data/MBDU/midla/notebooks/campcc/single_trial_betas/test_file.json'
trial_n = 3
condition_name = 'trial_type.word'
with open(json_orig_path, 'r') as h:
    jdat = json.load(h)
jdat["Steps"][0]['Model']['X'].remove(condition_name)
jdat["Steps"][0]['Model']['X'].append(f"trial-{trial_n:03d}_{condition_name}")
jdat["Steps"][0]['Model']['X'].append(f"other_trials_{condition_name}")

jdat['Steps'] = [jdat['Steps'][0]]
jdat['Steps'][0].pop('Contrasts', None)
jdat['Steps'][0].pop('DummyContrasts', None)

to_convolve = [x for x in jdat["Steps"][0]['Transformations'] if x["Name"] == "Convolve"][0]["Input"]
to_convolve.remove(condition_name)
to_convolve.append(f"trial-{trial_n:03d}_{condition_name}")
to_convolve.append(f"other_trials_{condition_name}")

[x for x in jdat["Steps"][0]['Transformations'] if x["Name"] == "Convolve"][0]["Input"] = to_convolve
jdat['Steps'][0]['DummyContrasts'] = {
    "Conditions": [f"trial-{trial_n:03d}_{condition_name}"],
    "Type":"t"
}
with open(json_new_path, 'w') as h:
    json.dump(jdat, h, indent=2)

In [32]:
jdat.keys()

dict_keys(['Name', 'Description', 'Input', 'Steps', 'DummyContrasts'])

In [33]:
json_new_path = '/data/MBDU/midla/notebooks/campcc/single_trial_betas/test_file_out.json'
trial_n = 3
condition_name = 'trial_type.word'
with open(json_orig_path, 'r') as h:
    jdat = json.load(h)

In [22]:
type(jdat['Steps'][0])

dict

In [20]:
jdat['Steps'] = [jdat['Steps'][0]]

In [36]:
#jdat['Steps'] = [jdat['Steps'][0]]
jdat['Steps'][0].pop('Contrasts', None)
jdat['Steps'][0].pop('DummyContrasts', None)

In [42]:
jdat['Steps'][0]['DummyContrasts'] = {
    "Conditions": [f"trial-{trial_n:03d}_{condition_name}"],
    "Type":"t"
}

In [43]:
jdat

{'Name': 'ds003_model001',
 'Description': '',
 'Input': {'task': 'rhymejudgment'},
 'Steps': [{'Level': 'run',
   'Transformations': [{'Name': 'Factor', 'Input': ['trial_type']},
    {'Name': 'Convolve',
     'Input': ['trial_type.word', 'trial_type.pseudoword'],
     'Model': 'spm'}],
   'Model': {'X': ['trial_type.word',
     'trial_type.pseudoword',
     'framewise_displacement',
     'trans_x',
     'trans_y',
     'trans_z',
     'rot_x',
     'rot_y',
     'rot_z',
     'a_comp_cor_00',
     'a_comp_cor_01',
     'a_comp_cor_02',
     'a_comp_cor_03',
     'a_comp_cor_04',
     'a_comp_cor_05']},
   'DummyContrasts': {'Conditions': ['trial-003_trial_type.word'],
    'Type': 't'}}]}

In [61]:
to_convolve = [x for x in jdat["Steps"][0]['Transformations'] if x["Name"] == "Convolve"][0]["Input"]
to_convolve.remove(condition_name)
to_convolve.append(f"trial-{trial_n:03d}_{condition_name}")
to_convolve.append(f"other_trials_{condition_name}")

In [127]:
! export PATH=/data/MBDU/midla/env/bin:$PATH; python /data/MBDU/midla/notebooks/campcc/single_trial_betas/BIDS_stb/bids_stb/stb.py /data/MBDU/midla/notebooks/campcc/single_trial_betas/test_file.json /data/MBDU/midla/notebooks/campcc/single_trial_betas/ trial_type.Lips 10

In [124]:
import click
import sys
import json
import os


@click.command()
@click.argument('json_orig_path', type=click.Path(exists=True)) #help='Path of the original BIDS-format JSON.')
@click.option('--condition_names','-c',multiple=True, help='Condition names as they appear in the BIDS file. Takes multipe inputs separated by -c.')
@click.option('--num_trials', '-n',  default=1, help='Number of trials. Defaults to 1')
@click.option('--json_new_dir','-d', default='.', help='Output directory for new JSONs. Defaults to current working directory.')
def stb_model(json_orig_path, condition_names, num_trials, json_new_dir):
	_stb(json_orig_path, condition_names, num_trials, json_new_dir)


def _stb(json_orig_path, condition_names, num_trials, json_new_dir):
    """Converts BIDS formatted jsons into jsons with single trial beta outputs.
    Takes original json path, target conditions, the number of total trials, and an optional output directory.
    Generates json for each trial of each condition with single trial beta output."""
    for condition_name in condition_names:
        for trial_n in range(num_trials):
            with open(json_orig_path, 'r') as h:
                jdat = json.load(h)

            #Replaces target condition with target trial and other trials regressors
            jdat['Steps'][0]['Model']['X'].remove(condition_name)
            jdat['Steps'][0]['Model']['X'] = [f'trial-{trial_n:03d}_{condition_name}',f'other_trials_{condition_name}'] + jdat['Steps'][0]['Model']['X']

            
            #Removes all model levels other than 1st level, contrasts, and existing dummy contrasts
            jdat['Steps'] = [jdat['Steps'][0]]
            jdat['Steps'][0].pop('Contrasts', None)
            jdat['Steps'][0].pop('DummyContrasts', None)

            #Adds transformations
            transform_list = jdat['Steps'][0]['Transformations']
            transform_list = [{"Name":"Factor","Input":["trial_number"]}] + transform_list
            new_transforms = [{
                  "Name": "And",
                  "Input": [
                    f"trial_number.{trial_n}",
                    condition_name
                  ],
                  "Output": [
                    f"trial-{trial_n:03d}_{condition_name}"
                  ]
                },            
                {
                  "Name": "Not",
                  "Input": [
                    f"trial_number.{trial_n}"
                  ],
                  "Output": [
                    "other_trials"
                  ]
                },
                {
                  "Name": "And",
                  "Input": [
                    "other_trials",
                    condition_name
                  ],
                  "Output": [
                    f"other_trials_{condition_name}"
                  ]
                }]
            try: 
                c_ind = [idx for idx,s in enumerate(transform_list) if s['Name'] == 'Convolve'][0]
                transform_list = transform_list[:c_ind] + new_transforms + transform_list[c_ind:]
            except IndexError:
                transform_list = transform_list + new_transforms
            jdat['Steps'][0]['Transformations'] = transform_list   
            
            
            #Adds target trial regressors to convolve list
            to_convolve = [x for x in jdat['Steps'][0]['Transformations'] if x['Name'] == 'Convolve'][0]['Input']
            to_convolve.remove(condition_name)
            to_convolve = [f'trial-{trial_n:03d}_{condition_name}',f'other_trials_{condition_name}'] + to_convolve
            [x for x in jdat['Steps'][0]['Transformations'] if x['Name'] == 'Convolve'][0]['Input'] = to_convolve

            #Adds target trial regressor to dummy contrast
            jdat['Steps'][0]['DummyContrasts'] = {
                'Conditions': [f'trial-{trial_n:03d}_{condition_name}'],
                'Type':'t'
            }
            #generates output json'''
            json_new_path = os.path.join(json_new_dir, f'stb_trial-{trial_n:03d}_{condition_name}.json')
            with open(json_new_path, 'w') as h:
                json.dump(jdat, h, indent=2)

In [125]:
json_orig_path = '/data/MBDU/midla/data/derivatives/fitlins/models/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-20900/ses-01/run-1/sub-20900_ses-01_task-mid_run-1_desc-model.json'
json_new_dir = '/data/MBDU/midla/notebooks/campcc/single_trial_betas/'
condition_names = ['gain_anticipation']
num_trials = 2
_stb(json_orig_path, condition_names, num_trials, json_new_dir)